# Pre-Margin Linking Dominating Contract Percentages

This notebook calculates some of the statistics used in the text associated with Figures 9 and 10.

In [1]:
import os
import sys
import re

from itertools import *
from operator import mul
from functools import reduce
import pickle

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

idx = pd.IndexSlice

from research_tools import storage

pd.options.display.float_format = lambda x: '{:,.4f}'.format(x) if abs(x) < 1 else '{:,.2f}'.format(x)

legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#555555', label='Yes Contracts', markersize=10),
    Line2D([0], [0], marker='X', color='w', markerfacecolor='#555555', label='No Contracts', markersize=10)
]

# Load Data

In [2]:
os.chdir('..')

basename = 'dem'

dem_replicating_contracts, = storage.retrieve_all([basename + '.replicating_contracts'])

basename = 'gop'

rep_replicating_contracts, = storage.retrieve_all([basename + '.replicating_contracts'])

Reading data from data/dem.replicating_contracts.p
Reading data from data/gop.replicating_contracts.p


In [3]:
dem_replicating_contracts.keys()

dict_keys(['yes contracts no spreads or fees', 'no contracts no spreads or fees', 'yes contracts spreads but no fees', 'no contracts spreads but no fees', 'no contracts spreads and fees', 'yes contracts spreads and fees'])

In [4]:
dem_trader_analysis1_yes = dem_replicating_contracts['yes contracts no spreads or fees']
dem_trader_analysis1_no = dem_replicating_contracts['no contracts no spreads or fees']

rep_trader_analysis1_yes = rep_replicating_contracts['yes contracts no spreads or fees']
rep_trader_analysis1_no = rep_replicating_contracts['no contracts no spreads or fees']

dem_trader_analysis2_yes = dem_replicating_contracts['yes contracts spreads but no fees']
dem_trader_analysis2_no = dem_replicating_contracts['no contracts spreads but no fees']

rep_trader_analysis2_yes = rep_replicating_contracts['yes contracts spreads but no fees']
rep_trader_analysis2_no = rep_replicating_contracts['no contracts spreads but no fees']

dem_trader_analysis3_yes = dem_replicating_contracts['yes contracts spreads and fees']
dem_trader_analysis3_no = dem_replicating_contracts['no contracts spreads and fees']

rep_trader_analysis3_yes = rep_replicating_contracts['yes contracts spreads and fees']
rep_trader_analysis3_no = rep_replicating_contracts['no contracts spreads and fees']

#  Limits

This notebook analyzes price data from the pre-margin linking period.

Note that very early in the life of this market, some of the trades took place before any market activity took place for other contracts. These trades cannot be replicated at all because the data needed to do so does not exist.

In [5]:
rep_margin_linking_cutoff_date = '2015-12-01'
dem_margin_linking_cutoff_date = '2015-10-22'

replication_share_qty_cutoff = 10

In [6]:
# only use trades after the dem margin linking cutoff date
dem_trader_analysis3_yes = dem_trader_analysis3_yes[
    dem_trader_analysis3_yes.loc[idx[:, ('_trades', 'date_executed')]] <= dem_margin_linking_cutoff_date].copy()
dem_trader_analysis3_no = dem_trader_analysis3_no[
    dem_trader_analysis3_no.loc[idx[:, ('_trades', 'date_executed')]] <= dem_margin_linking_cutoff_date].copy()

# only use trades after the rep margin linking cutoff date
rep_trader_analysis3_yes = rep_trader_analysis3_yes[
    rep_trader_analysis3_yes.loc[idx[:, ('_trades', 'date_executed')]] <= rep_margin_linking_cutoff_date].copy()
rep_trader_analysis3_no = rep_trader_analysis3_no[
    rep_trader_analysis3_no.loc[idx[:, ('_trades', 'date_executed')]] <= rep_margin_linking_cutoff_date].copy()

# Yes contracts, spreads and fees

In [7]:
dem_trader_analysis3_yes[('replication', 'dominated')] = (
    dem_trader_analysis3_yes[('_trades', 'price_per_share')] > dem_trader_analysis3_yes[('replication', 'max_loss')]
)

take_index = dem_trader_analysis3_yes[('_trades', 'take_provide')] == 'T'
provide_index = dem_trader_analysis3_yes[('_trades', 'take_provide')] == 'P'
dominated = dem_trader_analysis3_yes[('replication', 'dominated')]
not_na = ~pd.isnull(dem_trader_analysis3_yes[('replication', 'max_loss')])

In [8]:
(take_index & dominated & not_na).sum() / (take_index & not_na).sum()

1.0

In [9]:
(provide_index & dominated &  not_na).sum() / (provide_index & not_na).sum()

0.98717948717948723

In [10]:
rep_trader_analysis3_yes[('replication', 'dominated')] = (
    rep_trader_analysis3_yes[('_trades', 'price_per_share')] > rep_trader_analysis3_yes[('replication', 'max_loss')]
)

take_index = rep_trader_analysis3_yes[('_trades', 'take_provide')] == 'T'
provide_index = rep_trader_analysis3_yes[('_trades', 'take_provide')] == 'P'
dominated = rep_trader_analysis3_yes[('replication', 'dominated')]
not_na = ~pd.isnull(rep_trader_analysis3_yes[('replication', 'max_loss')])

In [11]:
(take_index & dominated & not_na).sum() / (take_index & not_na).sum()

0.99704579025110784

In [12]:
(provide_index & dominated &  not_na).sum() / (provide_index & not_na).sum()

0.99123287671232874

# No contracts, spreads and fees

In [13]:
def get_max_value(d_dict):
    try:
        d_dict = eval(d_dict)
        d_dict.pop('contracts')
        return max(d_dict.values())
    except:
        return np.nan

In [14]:
def get_contract_type(d_dict):
    try:
        d_dict = eval(d_dict)
        return d_dict.pop('contracts')
    except:
        return 'NEITHER'

In [15]:
dem_trader_analysis3_no[('replication', 'dominated_yes')] = (
    (dem_trader_analysis3_no[('_trades', 'price_per_share')] > dem_trader_analysis3_no[('replication', 'max_loss')]) &
    (dem_trader_analysis3_no[('replication', 'd')].apply(get_max_value) < replication_share_qty_cutoff) &
    (dem_trader_analysis3_no[('replication', 'd')].apply(get_contract_type) == 'YES')
)

dem_trader_analysis3_no[('replication', 'dominated_no')] = (
    (dem_trader_analysis3_no[('_trades', 'price_per_share')] > dem_trader_analysis3_no[('replication', 'max_loss')]) &
    (dem_trader_analysis3_no[('replication', 'd')].apply(get_max_value) < replication_share_qty_cutoff) &
    (dem_trader_analysis3_no[('replication', 'd')].apply(get_contract_type) == 'NO')
)

take_index = dem_trader_analysis3_no[('_trades', 'take_provide')] == 'T'
provide_index = dem_trader_analysis3_no[('_trades', 'take_provide')] == 'P'
dominated_yes = dem_trader_analysis3_no[('replication', 'dominated_yes')]
dominated_no = dem_trader_analysis3_no[('replication', 'dominated_no')]
not_na = ~pd.isnull(dem_trader_analysis3_no[('replication', 'max_loss')])

In [16]:
(take_index & dominated_yes & not_na).sum() / (take_index & not_na).sum()

0.0

In [17]:
(provide_index & dominated_yes & not_na).sum() / (provide_index & not_na).sum()

0.0

In [18]:
(take_index & dominated_no & not_na).sum() / (take_index & not_na).sum()

0.14316702819956617

In [19]:
(provide_index & dominated_no & not_na).sum() / (provide_index & not_na).sum()

0.15283842794759825

In [20]:
rep_trader_analysis3_no[('replication', 'dominated_yes')] = (
    (rep_trader_analysis3_no[('_trades', 'price_per_share')] > rep_trader_analysis3_no[('replication', 'max_loss')]) &
    (rep_trader_analysis3_no[('replication', 'd')].apply(get_max_value) < replication_share_qty_cutoff) &
    (rep_trader_analysis3_no[('replication', 'd')].apply(get_contract_type) == 'YES')
)

rep_trader_analysis3_no[('replication', 'dominated_no')] = (
    (rep_trader_analysis3_no[('_trades', 'price_per_share')] > rep_trader_analysis3_no[('replication', 'max_loss')]) &
    (rep_trader_analysis3_no[('replication', 'd')].apply(get_max_value) < replication_share_qty_cutoff) &
    (rep_trader_analysis3_no[('replication', 'd')].apply(get_contract_type) == 'NO')
)

take_index = rep_trader_analysis3_no[('_trades', 'take_provide')] == 'T'
provide_index = rep_trader_analysis3_no[('_trades', 'take_provide')] == 'P'
dominated_yes = rep_trader_analysis3_no[('replication', 'dominated_yes')]
dominated_no = rep_trader_analysis3_no[('replication', 'dominated_no')]
not_na = ~pd.isnull(rep_trader_analysis3_no[('replication', 'max_loss')])

In [21]:
(take_index & dominated_yes & not_na).sum() / (take_index & not_na).sum()

0.0

In [22]:
(provide_index & dominated_yes & not_na).sum() / (provide_index & not_na).sum()

0.0

In [23]:
(take_index & dominated_no & not_na).sum() / (take_index & not_na).sum()

0.97906602254428343

In [24]:
(provide_index & dominated_no & not_na).sum() / (provide_index & not_na).sum()

0.98626817447495962